<a href="https://colab.research.google.com/github/Svetorus/Recomen_systems/blob/master/RS_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420279 sha256=fb83ffffe9a127a5d44feded3590b843819697ae2caefb2d57191fabb05a8f6e
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [7]:
from google.colab import files
uploaded = files.upload()

Saving retail_train.csv to retail_train (1).csv


In [20]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [32]:
from google.colab import files
uploaded = files.upload()

Saving predictions_basic.csv to predictions_basic.csv


In [83]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [119]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [121]:
# Случайные рекоммендации
# Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1

def weighted_random_recommendation(items_weights, n=5):
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    proba = items_weights['weight'].to_list()

    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=proba)
    
    return recs.tolist()

In [122]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights = items_weights[items_weights['sales_value']>1]
items_weights['weight']= np.log(items_weights['sales_value'] + 0.001)
items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value', axis=1, inplace=True)

result['weighted_random_recommendation'] = result['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=5))

CPU times: user 7.78 s, sys: 57.9 ms, total: 7.84 s
Wall time: 7.85 s


In [123]:
result[['user_id','actual','weighted_random_recommendation']].head(3)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[876815, 827047, 1114006, 8205252, 831630]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[7168450, 848725, 837694, 5995312, 851398]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[973198, 1124874, 905986, 914545, 6602348]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [125]:
result = pd.read_csv('predictions_basic.csv')

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [126]:
# Конвертация строковых ячеек с список
def str_tolist(row):
   
    list_int = []
    for i in row.strip('][').split():
        list_int.append(int(i.rstrip(',')))
    return list_int

for cat_colname in result.select_dtypes(include='object').columns:
    result[cat_colname] = result.apply(lambda s: str_tolist(s[cat_colname]), axis=1)

In [127]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [128]:
for col in result.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()))

random_recommendation:  0.0006
popular_recommendation:  0.1552
itemitem:  0.0336
cosine:  0.0353
tfidf:  0.0361
own_purchases:  0.1800


лучшие предсказания:  
popular_recommendation 15% точности  
own_purchases          18% точности

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [116]:
# Улучшим бейзлайн присвоения весов товарам
# Ранжирование товаров по весам, пропорциональным логарифму продаж по топовым позициям
def get_popular_item_weights(df, ntop=5000):
    pop_weights = df.groupby('item_id')['sales_value'].sum().reset_index()
    pop_weights.sort_values('sales_value', ascending=False, inplace=True)

    pop_weights = pop_weights[:ntop]
    pop_weights['weight']= np.log(pop_weights['sales_value'] + 1e-3)
    pop_weights['weight'] = pop_weights['weight'] / pop_weights['weight'].sum()
    pop_weights.drop('sales_value', axis=1, inplace=True)
    
    return pop_weights

In [117]:
%%time

# Объединение Weighted Random Recommendation и Popularity-based recommendation
ntop_positions = 4000
pop_item_weights = get_popular_item_weights(data_train, ntop_positions)

result['popular_weighted_random_recommendation'] = result['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=k))
result.head()

CPU times: user 7.63 s, sys: 62.4 ms, total: 7.7 s
Wall time: 7.71 s


In [118]:
for col in result.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()))

random_recommendation:  0.0006
popular_recommendation:  0.1552
itemitem:  0.0336
cosine:  0.0353
tfidf:  0.0361
own_purchases:  0.1800
popular_weighted_random_recommendation:  0.0020
